# Bedrock Knowledge Base, S3 bucket and Open Search Index/Collection

In [ ]:
graph LR;
    A[Simple Local File] -->|SDK; Load To AWS S3| B[AWS S3];
    B-->|SDK; Ingest job run| C(Chunks);
    C-->D[Embeddings model];
    D-->|Store vectors|E[Vector Store / AWS OpenSearch];
    F[User query]-->D;
    D-->|User query results, similarity search|E;
    E-->|Found simlar vectors,\nAugment prompt|G[Standard LLM prompt context];

## Use AWS Cloud Shell or AWS CLI to create necessary resources

### Create S3 bucket
```
# list S3 buckets
aws s3 ls

aws s3 ls s3://bedrock-test-bucket > /dev/null 2>&1
if [ $? -eq 0 ]; then
    echo "Bucket 'bedrock-test-bucket' exists"
else
    echo "Bucket does not exist, creating"
    aws s3 mb s3://s3-quokkavec-bedrock-test --region us-east-1
fi
```

### Create a user group and user for Bedrock admin
 - Since Knowledge Base can't be made by "root" user, we'll need new IAM user with rights for KB creation in Bedrock
 - Make this by hand in Web UI for now
 - ~~Create "Access Key" for this user to be used in CloudShell~~
 - ~~Then "login" as that user in aws CloudShell~~
 - As of now 05.2024, AWS CLI doesn't have ability to make Knowledge Bases

```
aws configure # to configure user in CloudShell/CLI
# fill in access key, secret key etc., use default output format json
```

### TODO: creating role with execution rights for opensearch

### Create a vector store / OpenSearch Serverless Index
```
# aws opensearchserverless create-collection --name oss-quokkavec-bedrock-test
# see https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb
# easier way seem to click in in WebUI for now
```

## Cleanup
```
 aws s3 rm s3://s3-quokkavec-bedrock-test --region us-east-1
```

## Notes
 - For now I created KB in the Web Interface; TODO: CLI commands for all of it

In [ ]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: AWSSDK.BedrockRuntime, 3.7.302.7"
#r "nuget: AWSSDK.BedrockAgentRuntime, 3.7.302.7"
#r "nuget: Rockhead.Extensions, 0.1.2-preview"

# Use Access Keys and AWS/Bedrock SDK to run model with RAG

In [ ]:
using System.IO;
using Microsoft.Extensions.Configuration;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Amazon;
using Amazon.Util;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Amazon.BedrockAgent.Model;
using Amazon.BedrockAgentRuntime;
using Amazon.BedrockAgentRuntime.Model;

using Rockhead.Extensions; // TODO: consider contributing

// following is unfinished


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();

string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

string modelId = "meta.llama3-70b-instruct-v1:0";
AmazonBedrockAgentRuntimeClient bedrockAgentClient = new(awsAccessKeyId, awsSecretAccessKey, RegionEndpoint.USEast1);

bedrockAgentClient.Retrie